# Import everything

In [1]:
import csv
import torch
from tqdm.notebook import tqdm
from helper import loader, boilerplate
from settings.cfg import model, device
from settings.cfg import batch_size, tests_paths
from settings.cfg import model_path, result_path

Initialized


# Get the dataset


In [2]:
test_image, test_names = loader.load_data(tests_paths)

# Boilerplate Code

In [3]:
Data = boilerplate.ExportDataset

# Model

In [4]:
model.load_state_dict(torch.load(model_path))


RuntimeError: Error(s) in loading state_dict for NoeNet:
	Missing key(s) in state_dict: "increase.6.weight", "increase.6.bias". 
	Unexpected key(s) in state_dict: "increase.10.weight", "increase.10.bias", "increase.5.weight", "increase.5.bias", "increase.8.weight", "increase.8.bias", "classifier.12.weight", "classifier.12.bias". 
	size mismatch for steady.0.weight: copying a param with shape torch.Size([512, 512, 3, 3]) from checkpoint, the shape in current model is torch.Size([128, 128, 3, 3]).
	size mismatch for steady.0.bias: copying a param with shape torch.Size([512]) from checkpoint, the shape in current model is torch.Size([128]).
	size mismatch for steady.2.weight: copying a param with shape torch.Size([512, 512, 3, 3]) from checkpoint, the shape in current model is torch.Size([128, 128, 3, 3]).
	size mismatch for steady.2.bias: copying a param with shape torch.Size([512]) from checkpoint, the shape in current model is torch.Size([128]).
	size mismatch for steady.4.weight: copy

# Run tests

In [ ]:
tests = Data(test_image, test_names)
tests_dataloader = torch.utils.data.DataLoader(tests, batch_size=batch_size)

with torch.no_grad(), open(result_path, "w", newline='') as result:
    result_writer = csv.writer(result)
    result_writer.writerow(["Frame", "Label"])
    for image, names in tqdm(tests_dataloader):
        image = image.to(device, dtype=torch.float)

        output = model(image)
        output = output.argmax(dim=1).cpu().numpy()
        assert(output.shape[0] == len(names))
        
        result_writer.writerows(zip(names, output))


# Submit


In [ ]:
from datetime import datetime
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

api.competition_submit(result_path, f"Submitted at {datetime.now()}", "hsgs-hackathon2022")
